<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/9h_Multiple_Parallel_Input_and_MultiStep_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiple Parallel Input and Multi-Step Output**

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out): 
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the dataset 
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential 
from keras.layers import Dense

**The input sequence**

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1)) 

**Create a dataset**

In [ ]:
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

In [ ]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

In [ ]:
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out) 
print(X.shape, y.shape)

In [ ]:
# summarize the data
for i in range(len(X)):
  print("\nX:", X[i],"\n\ny:", y[i])

In [ ]:
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input)) # flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))
y

**Create the model**

In [ ]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input)) 
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

**Train the model**

In [ ]:
model.fit(X, y, epochs=2000, verbose=0)

**Enter values for a prediction**

We expect values to be:<br>
90, 95, 185<br>
100, 105, 205

In [ ]:
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]]) 
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)